In [22]:
import os

IS_KAGGLE = bool(os.environ.get("KAGGLE_KERNEL_RUN_TYPE"))
print(f"running on kaggle: {IS_KAGGLE}")

running on kaggle: False


In [23]:
if not IS_KAGGLE:
    !pip install --upgrade numpy pandas xgboost scikit-learn
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    !pip install \
        --extra-index-url=https://pypi.nvidia.com \
        "cudf-cu12==25.2.*" "dask-cudf-cu12==25.2.*" "cuml-cu12==25.2.*" \
        "cugraph-cu12==25.2.*" "nx-cugraph-cu12==25.2.*" "cuspatial-cu12==25.2.*" \
        "cuproj-cu12==25.2.*" "cuxfilter-cu12==25.2.*" "cucim-cu12==25.2.*" \
        "pylibraft-cu12==25.2.*" "raft-dask-cu12==25.2.*" "cuvs-cu12==25.2.*" \
        "nx-cugraph-cu12==25.2.*"

!pip install gputil

  Using cached numpy-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Looking in indexes: https://download.pytorch.org/whl/cu121
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [24]:
import warnings

warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import xgboost as xgb
import torch
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import cudf
import GPUtil

if torch.cuda.is_available():
    device = "cuda"
else:
    raise

datasets_dir = "../input" if IS_KAGGLE else "../datasets"
kfold = KFold(shuffle=True, random_state=42)

In [25]:
fn = "train_poss.csv"
print(f"reading {fn}")
train = pd.read_csv(f"{datasets_dir}/net-dataset/{fn}")

train = pd.concat(
    [
        train.select_dtypes("int64").astype("int32"),
        train.select_dtypes("float64").astype("float32"),
    ],
    axis=1,
)

reading train_poss.csv


In [26]:
print("train")
int_cols = train.select_dtypes("int32").columns.to_list()
print(f"{len(int_cols):>3} {int_cols}")
float_cols = train.select_dtypes("float32").columns.to_list()
o_cols = [c for c in float_cols if c.split("_")[2] == "o"]
print(f"{len(o_cols):>3} {o_cols[:3]} ... {o_cols[-3:]}")
d_cols = [c for c in float_cols if c.split("_")[2] == "d"]
print(f"{len(d_cols):>3} {d_cols[:3]} ... {d_cols[-3:]}")
sos_o_cols = [c for c in float_cols if c.split("_")[3] == "o"]
print(f"{len(sos_o_cols):>3} {sos_o_cols[:3]} ... {sos_o_cols[-3:]}")
sos_d_cols = [c for c in float_cols if c.split("_")[3] == "d"]
print(f"{len(sos_d_cols):>3} {sos_d_cols[:3]} ... {sos_d_cols[-3:]}")
print("---")
print(
    f"{train.shape[1]} {len(int_cols) + len(o_cols) + len(d_cols) + len(sos_o_cols) + len(sos_d_cols)}"
)

train
  5 ['Season', 'DayNum', 'TeamID_1', 'TeamID_2', 'Margin']
 28 ['Score_poss_o_1', 'FGM_poss_o_1', 'FGA_poss_o_1'] ... ['Stl_poss_o_2', 'Blk_poss_o_2', 'PF_poss_o_2']
 28 ['Score_poss_d_1', 'FGM_poss_d_1', 'FGA_poss_d_1'] ... ['Stl_poss_d_2', 'Blk_poss_d_2', 'PF_poss_d_2']
 28 ['sos_Score_poss_o_1', 'sos_FGM_poss_o_1', 'sos_FGA_poss_o_1'] ... ['sos_Stl_poss_o_2', 'sos_Blk_poss_o_2', 'sos_PF_poss_o_2']
 28 ['sos_Score_poss_d_1', 'sos_FGM_poss_d_1', 'sos_FGA_poss_d_1'] ... ['sos_Stl_poss_d_2', 'sos_Blk_poss_d_2', 'sos_PF_poss_d_2']
---
117 117


In [27]:
print(f"train: {str(train.shape):>23}")

X_df = train.drop(columns=["Season", "DayNum", "TeamID_1", "TeamID_2", "Margin"])
print(f"X_df: {str(X_df.shape):>24}")

y_s = train["Margin"]
print(f"y_s: {str(y_s.shape):>21}")
scaler_y = StandardScaler()

train:           (202033, 117)
X_df:            (202033, 112)
y_s:             (202033,)


In [28]:
def brier_score(y_pred_oof):
    win_prob_pred_oof = 1 / (1 + np.exp(-y_pred_oof * 0.1))
    team_1_won = (y_s > 0).astype("int32")
    return np.mean((win_prob_pred_oof - team_1_won) ** 2)

In [29]:
print(f"xgboost")
y_pred_oof = np.zeros(y_s.shape[0])

for fold_n, (i_fold, i_oof) in enumerate(kfold.split(X_df.index), 1):
    print(f"  fold {fold_n}")
    m = xgb.XGBRegressor(
        tree_method="hist",
        device="cuda",
        max_depth=3,
        colsample_bytree=0.5,
        subsample=0.8,
        n_estimators=2000,
        learning_rate=0.02,
        min_child_weight=80,
        verbosity=1,
    )

    X_fold = cudf.DataFrame.from_pandas(X_df.iloc[i_fold])
    y_fold = cudf.Series(y_s.iloc[i_fold])
    X_oof = cudf.DataFrame.from_pandas(X_df.iloc[i_oof])
    y_oof = cudf.Series(y_s.iloc[i_oof])

    m.fit(
        X_fold,
        y_fold,
        verbose=500,
        eval_set=[(X_fold, y_fold), (X_oof, y_oof)],
    )

    y_pred_oof[i_oof] = m.predict(X_oof)
    GPUtil.showUtilization()
    print()

score = brier_score(y_pred_oof)
print(f"xgboost score: {score:.4f}")

xgboost
  fold 1
[0]	validation_0-rmse:16.40764	validation_1-rmse:16.48630
[500]	validation_0-rmse:11.23856	validation_1-rmse:11.36118
[1000]	validation_0-rmse:10.97439	validation_1-rmse:11.14115
[1500]	validation_0-rmse:10.89099	validation_1-rmse:11.09311
[1999]	validation_0-rmse:10.83894	validation_1-rmse:11.07478
| ID | GPU | MEM |
------------------
|  0 | 63% | 13% |

  fold 2
[0]	validation_0-rmse:16.41837	validation_1-rmse:16.44564
[500]	validation_0-rmse:11.25720	validation_1-rmse:11.29307
[1000]	validation_0-rmse:10.99616	validation_1-rmse:11.06826
[1500]	validation_0-rmse:10.91148	validation_1-rmse:11.02228
[1999]	validation_0-rmse:10.85869	validation_1-rmse:11.00443
| ID | GPU | MEM |
------------------
|  0 | 34% | 13% |

  fold 3
[0]	validation_0-rmse:16.42853	validation_1-rmse:16.40116
[500]	validation_0-rmse:11.24714	validation_1-rmse:11.36190
[1000]	validation_0-rmse:10.98022	validation_1-rmse:11.13392
[1500]	validation_0-rmse:10.89689	validation_1-rmse:11.08670
[1999]	

In [30]:
print("torch")

X = torch.as_tensor(
    StandardScaler().fit_transform(X_df.values),
    dtype=torch.float32,
    device=device,
)
print(f"X:    {X.shape}")

y = torch.tensor(
    scaler_y.fit_transform(train[["Margin"]]).flatten(),
    dtype=torch.float32,
    device=device,
)
print(f"y:    {y.shape}")

n_epochs = 1_000
hidden_size = 64
loss_fn = torch.nn.MSELoss()

y_pred_oof = torch.zeros(
    y.shape[0],
    dtype=torch.float32,
    requires_grad=False,
    device=device,
)

for fold_n, (i_fold, i_oof) in enumerate(kfold.split(X_df.index), 1):
    print(f"  fold {fold_n}")

    weights1 = torch.nn.Parameter(
        0.1 * torch.randn(X_df.shape[1], hidden_size, device=device)
    )
    bias1 = torch.nn.Parameter(torch.zeros(hidden_size, device=device))
    weights2 = torch.nn.Parameter(0.1 * torch.randn(hidden_size, 1, device=device))
    bias2 = torch.nn.Parameter(torch.zeros(1, device=device))
    optimizer = torch.optim.Adam([weights1, bias1, weights2, bias2], weight_decay=1e-4)

    for epoch_n in range(1, n_epochs + 1):
        y_pred_fold_epoch = (
            F.leaky_relu(X[i_fold] @ weights1 + bias1, negative_slope=0.1) @ weights2
            + bias2
        )
        loss_fold_epoch = loss_fn(y_pred_fold_epoch, y[i_fold].view(-1, 1))
        optimizer.zero_grad()
        loss_fold_epoch.backward()
        optimizer.step()

        with torch.no_grad():
            y_pred_oof_epoch = (
                F.leaky_relu(X[i_oof] @ weights1 + bias1, negative_slope=0.1) @ weights2
                + bias2
            )
            loss_oof_epoch = loss_fn(y_pred_oof_epoch, y[i_oof].view(-1, 1))

        if epoch_n > (n_epochs - 3):
            print(
                f"    epoch {epoch_n:>6}: "
                f"fold={loss_fold_epoch.item():.4f} "
                f"oof={loss_oof_epoch.item():.4f}"
            )

    with torch.no_grad():
        y_pred_oof[i_oof] = (
            F.leaky_relu(X[i_oof] @ weights1 + bias1, negative_slope=0.1) @ weights2
            + bias2
        ).flatten()

    GPUtil.showUtilization()
    print()

y_pred_oof = scaler_y.inverse_transform(
    y_pred_oof.cpu().numpy().reshape(-1, 1)
).flatten()

score = brier_score(y_pred_oof)
print(f"torch score:   {score:.4f}")

torch
X:    torch.Size([202033, 112])
y:    torch.Size([202033])
  fold 1
    epoch    998: fold=0.4317 oof=0.4464
    epoch    999: fold=0.4316 oof=0.4464
    epoch   1000: fold=0.4316 oof=0.4464
| ID | GPU | MEM |
------------------
|  0 | 87% | 13% |

  fold 2
    epoch    998: fold=0.4328 oof=0.4398
    epoch    999: fold=0.4327 oof=0.4399
    epoch   1000: fold=0.4327 oof=0.4398
| ID | GPU | MEM |
------------------
|  0 | 84% | 13% |

  fold 3
    epoch    998: fold=0.4320 oof=0.4436
    epoch    999: fold=0.4320 oof=0.4436
    epoch   1000: fold=0.4320 oof=0.4436
| ID | GPU | MEM |
------------------
|  0 | 88% | 13% |

  fold 4
    epoch    998: fold=0.4344 oof=0.4425
    epoch    999: fold=0.4344 oof=0.4425
    epoch   1000: fold=0.4344 oof=0.4425
| ID | GPU | MEM |
------------------
|  0 | 88% | 13% |

  fold 5
    epoch    998: fold=0.4367 oof=0.4428
    epoch    999: fold=0.4367 oof=0.4428
    epoch   1000: fold=0.4367 oof=0.4428
| ID | GPU | MEM |
------------------
|  0 